train_set_file_name = "task1.train.txt"
#test_set_file_name ="test.txt"

In [1]:
train_set_file_name = "task1.train.txt"

#test_set_file_name ="test.txt"

In [2]:
import sys
import xgboost as xgb
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [3]:

article_contents,gold_labels = ([],[])
article_len = 0
with open(train_set_file_name, "r", encoding='utf-8') as f:
    for line in f.readlines():
        article_content, article_id, gold_label = line.rstrip().split("\t")
        article_len += len(article_content.split(' '))
        article_contents.append(article_content)
        gold_labels.append(gold_label)
print("Number of documents in the traning set: %d"%(len(article_contents)))
print("Number of labels in the training set: %d"%(len(gold_labels)))
print("Number of propaganda articles: %d"%(gold_labels.count('propaganda')))

print(set(gold_labels))
print("average length of an article is %f" % (article_len/len(article_contents)))

Number of documents in the traning set: 35993
Number of labels in the training set: 35993
Number of propaganda articles: 4021
{'non-propaganda', 'propaganda'}
average length of an article is 594.953769


In [4]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(article_contents, gold_labels, test_size=0.1)
print("Number of articles in training dataset: %d"%(len(x_train)))
print("Number of propaganda articles in training dataset: %d"%(y_train.count('propaganda')))
print("Number of articles in testing dataset: %d"%(len(x_test)))
print("Number of propaganda articles in testing dataset: %d"%(y_test.count('propaganda')))

Number of articles in training dataset: 32393
Number of propaganda articles in training dataset: 3625
Number of articles in testing dataset: 3600
Number of propaganda articles in testing dataset: 396


In [5]:
vectorizer = TfidfVectorizer(min_df=3, max_df=0.5, max_features=None, 
                             strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}', 
                             ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english')

train = vectorizer.fit_transform(x_train)
print("n_samples: %d, n_features: %d" % train.shape)
test = vectorizer.transform(x_test)

print("Checking that the number of features in train and test correspond: %s - %s" % (train.shape[1],test.shape[1]))
#print(type(train))

n_samples: 32393, n_features: 800864
Checking that the number of features in train and test correspond: 800864 - 800864


In [8]:
import re
cnt = 0
for article in x_train:
    pattern = re.compile(r'(\. )|(\? )|(\! )')
    res = pattern.findall(article)
    cnt += len(res)
    
print((cnt/len(x_train)))

24.38215046460655


In [16]:
#SVM
model = LinearSVC(C=1.0, class_weight='balanced', multi_class='ovr', random_state=40)
model.fit(train, y_train)
predictions = model.predict(test)
from sklearn.metrics import precision_recall_fscore_support
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.98227612 0.91927083]
recall: [0.99028213 0.86097561]
fscore: [0.98626288 0.88916877]
support: [3190  410]
accuracy: 0.9755555555555555


In [17]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression 
 
classifier = LogisticRegression() 
classifier.fit(train, y_train)  

predictions = classifier.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.94838902 0.95564516]
recall: [0.99655172 0.57804878]
fscore: [0.97187404 0.72036474]
support: [3190  410]
accuracy: 0.9488888888888889


In [18]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(oob_score=True, n_estimatsors = 100)
rf.fit(train, y_train)
predictions = rf.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.92571097 1.        ]
recall: [1.         0.37560976]
fscore: [0.96142254 0.54609929]
support: [3190  410]
accuracy: 0.9288888888888889


In [13]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
best_k=-1
best_score=0
best_method=''
for method in ['uniform','distance']:
    for i in range(1,11):
        knn_clf=KNeighborsClassifier(n_neighbors=i,weights=method)
        knn_clf.fit(train,y_train)
        scores=knn_clf.score(test,y_test)
        if scores>best_score:
            best_score=scores
            best_k=i
            best_method=method

print('best k:%d, best score:%.4f, best method: %s'%(best_k,best_score,best_method))

best k:6, best score:0.9614, best method: distance


In [15]:
knn_clf=KNeighborsClassifier(n_neighbors=6,weights='distance')
knn_clf.fit(train,y_train)
predictions = knn_clf.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.96466646 0.92744479]
recall: [0.99278997 0.71707317]
fscore: [0.97852619 0.8088033 ]
support: [3190  410]
accuracy: 0.9613888888888888


In [25]:
#Use article summaries to do classification
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
#from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.summarizers.luhn import LuhnSummarizer as Summarizer
#from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [10]:
train_output_file = '../../data/task_1/train.txt'
dev_output_file = '../../data/task_1/dev.txt'
test_output_file = '../../data/task_1/test.txt'
x_train, x_test, y_train, y_test = train_test_split(article_contents, gold_labels, test_size=0.2)
x_test, x_dev, y_test, y_dev = train_test_split(x_test, y_test, test_size=0.1)
print("Number of articles in training dataset: %d"%(len(x_train)))
print("Number of propaganda articles in training dataset: %d"%(y_train.count('propaganda')))
print("Number of articles in testing dataset: %d"%(len(x_dev)))
print("Number of propaganda articles in testing dataset: %d"%(y_dev.count('propaganda')))
print("Number of articles in testing dataset: %d"%(len(x_test)))
print("Number of propaganda articles in testing dataset: %d"%(y_test.count('propaganda')))


Number of articles in training dataset: 28794
Number of propaganda articles in training dataset: 3201
Number of articles in testing dataset: 720
Number of propaganda articles in testing dataset: 74
Number of articles in testing dataset: 6479
Number of propaganda articles in testing dataset: 746


In [26]:
from nltk.tokenize import word_tokenize

LANGUAGE = "english"
SENTENCES_COUNT = 5
stemmer = Stemmer(LANGUAGE)
#parser = PlaintextParser.from_string("Check this out.", Tokenizer(LANGUAGE))
summarizer = Summarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)


In [17]:
test_output_file = '../../data/task_1/test.txt'
with open(test_output_file,'w') as w:
    for (content, label) in zip(x_test, y_test):
        parser = PlaintextParser.from_string(content, Tokenizer(LANGUAGE))
        summary = " ".join([str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT)])
        tokens = word_tokenize(summary)
        w.write("\nlabel {}\n".format(label))
        for token in word_tokenize(summary):
            w.write("{}\n".format(token))

In [27]:
x_train_sum = []
x_test_sum = []

for content in x_train:
    parser = PlaintextParser.from_string(content, Tokenizer(LANGUAGE))
    summary = " ".join([str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT)])
    x_train_sum.append(summary)

for content in x_test:
    parser = PlaintextParser.from_string(content, Tokenizer(LANGUAGE))
    summary = " ".join([str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT)])
    x_test_sum.append(summary)
    
train_sum = vectorizer.transform(x_train_sum)
test_sum = vectorizer.transform(x_test_sum)
#print(test)
#feature_name = vectorizer.get_feature_names()
print("Checking that the number of features in train_sum and test_sum correspond: %s - %s" % (train.shape[1],test.shape[1]))
#print(type(train))

Checking that the number of features in train_sum and test_sum correspond: 800864 - 800864


In [28]:
#SVM_sum
model = LinearSVC(C=1.0, class_weight='balanced', multi_class='ovr', random_state=40)
model.fit(train_sum, y_train)
predictions = model.predict(test_sum)
from sklearn.metrics import precision_recall_fscore_support
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.96375921 0.80813953]
recall: [0.97940075 0.7020202 ]
fscore: [0.97151703 0.75135135]
support: [3204  396]
accuracy: 0.9488888888888889


In [10]:
#KNN_sum
from sklearn.neighbors import KNeighborsClassifier
best_k=-1
best_score=0
best_method=''
knn_clf=KNeighborsClassifier(n_neighbors=6,weights='distance')
knn_clf.fit(train_sum,y_train)
predictions = knn_clf.predict(test_sum) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.94300059 0.89252336]
recall: [0.99284826 0.49739583]
fscore: [0.96728264 0.63879599]
support: [3216  384]
accuracy: 0.94


In [11]:
#LogisticRegression_sum
from sklearn.linear_model import LogisticRegression 
 
classifier = LogisticRegression() 
classifier.fit(train_sum, y_train)  

predictions = classifier.predict(test_sum) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.92239149 0.94214876]
recall: [0.99782338 0.296875  ]
fscore: [0.95862584 0.45148515]
support: [3216  384]
accuracy: 0.9230555555555555


In [13]:
#RandomForest_sum
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(oob_score=True, n_estimators = 100)
rf.fit(train_sum, y_train)
predictions = rf.predict(test_sum) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.90950226 1.        ]
recall: [1.         0.16666667]
fscore: [0.95260664 0.28571429]
support: [3216  384]
accuracy: 0.9111111111111111
